In [11]:
!pip install python-dotenv


In [10]:
import os

from dotenv import load_dotenv
from langchain.chains import LLMChain, SequentialChain
from langchain.chat_models import AzureChatOpenAI
from langchain.prompts import PromptTemplate

load_dotenv()

# print(f"model: {os.environ['AZURE_OPENAI_35_TURBO_MODEL']}, \
# version: {os.environ['AZURE_OPENAI_35_TURBO_MODEL_VERSION']}, \
# deployment: {os.environ['AZURE_OPENAI_35_TURBO_DEPLOYMENT_ID']}, \
# endpoint: {os.environ['AZURE_OPENAI_API_BASE']}, \
# api key: {os.environ['AZURE_OPENAI_API_KEY']}")

# print(f"model: {os.environ['AZURE_OPENAI_4_TURBO_MODEL']}, \
# version: {os.environ['AZURE_OPENAI_4_TURBO_MODEL_VERSION']}, \
# deployment: {os.environ['AZURE_OPENAI_4_TURBO_DEPLOYMENT_ID']}, \
# endpoint: {os.environ['AZURE_OPENAI_API_BASE']}, \
# api key: {os.environ['AZURE_OPENAI_API_KEY']}")
# 

# till i add em to conda
os.environ['AZURE_OPENAI_35_16k_MODEL_VERSION']="2023-05-15"
os.environ['AZURE_OPENAI_35_16k_MODEL']="gpt-35-turbo"


os.environ['AZURE_OPENAI_EMBEDDINGS_DEPLOYMENT']='fcdo-digital-triage-embeddings'
os.environ['EMBEDDING_MODEL']='text-embedding-ada-002'

azureChatClient = AzureChatOpenAI(
    temperature=0,
    model=os.environ["AZURE_OPENAI_35_TURBO_MODEL"],
    openai_api_version=os.environ["AZURE_OPENAI_35_TURBO_MODEL_VERSION"],
    openai_api_key=os.environ["AZURE_OPENAI_API_KEY"],
    azure_endpoint=os.environ["AZURE_OPENAI_API_BASE"],
    deployment_name=os.environ["DEPLOYMENT_ID"],
)
# azureChatClient = AzureChatOpenAI(
#     temperature=0,
#     model=os.environ["AZURE_OPENAI_4_TURBO_MODEL"],
#     openai_api_version=os.environ["AZURE_OPENAI_4_TURBO_MODEL_VERSION"],
#     openai_api_key=os.environ["AZURE_OPENAI_API_KEY"],
#     azure_endpoint=os.environ["AZURE_OPENAI_API_BASE"],
#     deployment_name=os.environ["AZURE_OPENAI_4_TURBO_DEPLOYMENT_ID"],
# )


def brainstorm_phase():
    prompt = PromptTemplate(
        input_variables=["input", "perfect_factors"],
        template="You are a bot for the UK FCDO govenment department that will help UK citizens answer questions when they are abroad outside of the UK."
    )

    chain1 = LLMChain(
        llm=azureChatClient,
        prompt=prompt,
        output_key="solutions",
        verbose=True
    )
    return chain1


def orchestrate(query: str):
    overall_chain = SequentialChain(
        chains=[brainstorm_phase()],
        input_variables=["input", "perfect_factors"],
        output_variables=["ranked_solutions"],
        verbose=True
    )

    result = overall_chain({"input": query,
                            "perfect_factors": "you can contact your local embassy or apply for an emergency travel document to get home.  You can contact a local medical facility and report the incident to the police for the mugging."})

    print(result)

    # print(overall_chain({"input": "I've a UK citizen and lost my passport in bangkok and have been mugged",
    #                      "perfect_factors": "you can contact your local embassy or apply for an emergency travel document to get home.  You can contact a local medical facility and report the incident to the police for the mugging."}))
    return result


orchestrate("please help me, I've a UK citizen and lost my passport in bangkok and have been mugged")

KeyError: 'AZURE_OPENAI_35_TURBO_MODEL'